In [4]:
import pandas as pd
from io import StringIO

Load data from https://www.ipcc.ch/site/assets/uploads/2018/02/ipcc_wg3_ar5_annex-iii.pdf#page=7 which is in gCO2eq/kWh

Note, 1 gCO2/kWh = 1 kgCO2/MWh

In [33]:
# Parsing data from the IPCC report using https://tableconvert.com/markdown-to-csv

EMISSION_FACTORS_CSV = """
,Direct emissions,Infrastructure & supply chain emissions,Biogenic C02 emissions and albedo effect,Methane emissions,Lifecycle emissions (incl. albedo effect)
Options,Min/Median/Max,,Typical values,,Min/Median/Max
Currently Commercially Available Technologies,,,,,
Coal—PC,670/760/870,9.6,0,47,740/820/910
Gas—Combined Cycle,350/370/490,1.6,0,91,410/490/650
Biomass—cofiring,n. a.,,,,620/740/890
Biomass—dedicated,n.a.,210,27,0,130/230/420
Geothermal,0,45,0,0,6.0/38/79
Hydropower,0,19,0,88,1.0/24/2200
Nuclear,0,18,0,0,3.7/12/110
Concentrated Solar Power,0,29,0,0,8.8/27/63
Solar PV—rooftop,0,42,0,0,/41/60
Solar PV—utility,0,66,0,0,18/48/180
Wind onshore,0,15,0,0,7.0/11/56
Wind offshore,0,17,0,0,8.0/12/35
Pre-commercial Technologies,,,,,
ccs—Coal—Oxyfuel,14/76/1 10,17,0,67,100/160/200
ccs—Coal—PC,95/120/140,28,0,68,190/220/250
ccs—Coal—IGCC,100/120/150,9.9,0,62,170/200/230
ccs—Gas—Combined Cycle,30/57/98,8.9,0,110,94/170/340
Ocean,0,17,0,0,5.6/17/28
""".strip()

In [34]:
df = pd.read_csv(StringIO(EMISSION_FACTORS_CSV), index_col=0)
df

,Direct emissions,Infrastructure & supply chain emissions,Biogenic C02 emissions and albedo effect,Methane emissions,Lifecycle emissions (incl. albedo effect)
Options,Min/Median/Max,NaN,Typical values,NaN,Min/Median/Max
Currently Commercially Available Technologies,NaN,NaN,NaN,NaN,NaN
Coal—PC,670/760/870,9.6,0,47.0,740/820/910
Gas—Combined Cycle,350/370/490,1.6,0,91.0,410/490/650
Biomass—cofiring,n. a.,NaN,NaN,NaN,620/740/890
Biomass—dedicated,n.a.,210.0,27,0.0,130/230/420
Geothermal,0,45.0,0,0.0,6.0/38/79
Hydropower,0,19.0,0,88.0,1.0/24/2200
Nuclear,0,18.0,0,0.0,3.7/12/110
Concentrated Solar Power,0,29.0,0,0.0,8.8/27/63


In [47]:
emission_factors = df\
    .drop(labels=['Options', 'Currently Commercially Available Technologies', 'Pre-commercial Technologies'], axis=0)\
    .map(str)\
    .map(lambda x: x.split("/")[1] if "/" in x else x)\
    .map(lambda x: 0 if x in ["n.a.", "n. a.", "nan"] else x)\
    .map(float)\
    .apply(sum, axis=1)\
    .sort_index()
emission_factors

Biomass—cofiring             740.0
Biomass—dedicated            467.0
Coal—PC                     1636.6
Concentrated Solar Power      56.0
Gas—Combined Cycle           952.6
Geothermal                    83.0
Hydropower                   131.0
Nuclear                       30.0
Ocean                         34.0
Solar PV—rooftop              83.0
Solar PV—utility             114.0
Wind offshore                 29.0
Wind onshore                  26.0
ccs—Coal—IGCC                391.9
ccs—Coal—Oxyfuel             320.0
ccs—Coal—PC                  436.0
ccs—Gas—Combined Cycle       345.9
dtype: float64

In [48]:
name_mapping = {
    "biomass_mwh": ["Biomass—cofiring", "Biomass—dedicated"],
    "hydropower_mwh": ["Hydropower"],
    "wind_offshore_mwh": ["Wind offshore"],
    "wind_onshore_mwh": ["Wind onshore"],
    "photovoltaic_mwh": ["Solar PV—rooftop", "Solar PV—utility", 'Concentrated Solar Power'],
    "nuclear_mwh": ["Nuclear"],
    "brown_coal_mwh": ["Coal—PC"],  # Take coal here
    "hard_coal_mwh": ["Coal—PC"],  # Take coal here
    "natural_gas_mwh": ["Gas—Combined Cycle"],
    "pumped_storage_mwh": ["Hydropower", 'Ocean'],
    "other_conventional_mwh": ['Coal—PC', 'Gas—Combined Cycle'],  # Just take the average of the two conventional technologies
    "other_renewables_mwh": ['Geothermal']
}

# Make sure we do not miss any important entry
set(emission_factors.index) - {n for e in name_mapping.values() for n in e}

{'ccs—Coal—IGCC', 'ccs—Coal—Oxyfuel', 'ccs—Coal—PC', 'ccs—Gas—Combined Cycle'}

In [53]:
def mean(values):
    return sum(values) / len(values)

emission_factors_dict = {
    new_name: mean([
        emission_factors.loc[original_name] 
        for original_name in original_names
    ])
    for new_name, original_names in name_mapping.items()
    for name in original_names
}
emission_factors_dict

{'biomass_mwh': 603.5,
 'hydropower_mwh': 131.0,
 'wind_offshore_mwh': 29.0,
 'wind_onshore_mwh': 26.0,
 'photovoltaic_mwh': 84.33333333333333,
 'nuclear_mwh': 30.0,
 'brown_coal_mwh': 1636.6,
 'hard_coal_mwh': 1636.6,
 'natural_gas_mwh': 952.6,
 'pumped_storage_mwh': 82.5,
 'other_conventional_mwh': 1294.6,
 'other_renewables_mwh': 83.0}